In [3]:
!pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=04c230b201096557f3f125a9e739bc4f004f4c16cc31d48e59b95e645bdd11e6
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [89]:
import yfinance as yf
import pandas as pd
from ta.momentum import RSIIndicator
from ta.trend import SMAIndicator, MACD
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import joblib

In [95]:
def fetch_stock_data(ticker, period, interval):
    df = yf.download(ticker, period=period, interval=interval, group_by='ticker', auto_adjust=True)
    if ticker in df.columns.get_level_values(0):
        df = df[ticker].copy()
    else:
        df = df.loc[:, pd.IndexSlice[:, ticker]].copy()
        df.columns = df.columns.droplevel(1)
    return df

In [96]:
tickers = ['RELIANCE.NS','ITC.NS','TCS.NS']

In [97]:
results = []
for ticker in tickers:
    df = fetch_stock_data(ticker, period='6mo', interval='1d').copy()

    # Computing indicators
    df['RSI'] = RSIIndicator(df['Close'], window=14).rsi()
    df['SMA20'] = SMAIndicator(df['Close'], window=20).sma_indicator()
    df['SMA50'] = SMAIndicator(df['Close'], window=50).sma_indicator()
    macd = MACD(df['Close'], window_slow=26, window_fast=12, window_sign=9)
    df['MACD'] = macd.macd()
    df['MACD_Signal'] = macd.macd_signal()
    df = df.dropna(subset=['RSI','SMA20','SMA50','MACD','MACD_Signal','Volume']).copy()

    # Create target
    df.loc[:, 'target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
    df = df.iloc[:-1]

    # 5) features and labels
    features = ['RSI', 'SMA20', 'SMA50', 'MACD', 'MACD_Signal', 'Volume']
    X = df[features]
    y = df['target']

    # Scale and split
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, shuffle=False)

    # Train, evaluate, save
    model = DecisionTreeClassifier(random_state=42)
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    acc = accuracy_score(y_test, pred)
    results.append((ticker, acc))
    joblib.dump(model, f'{ticker}_model.joblib')
    joblib.dump(scaler, f'{ticker}_scaler.joblib')

# 8) Results
for ticker, accuracy in results:
    print(f"\nAccuracy for {ticker}: {accuracy:.4f}")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Accuracy for RELIANCE.NS: 0.4667

Accuracy for ITC.NS: 0.6667

Accuracy for TCS.NS: 0.4667
